In [ ]:
# !pip install imageio

In [ ]:
# !python train.py -h

In [ ]:
# files modified
"""
Pluralistic/train.py
Pluralistic/options/base_options.py
Pluralistic/dataloader/data_loader.py
"""
--continue_train

In [ ]:
from object_detect.generate_masks import create_rand_mask

In [ ]:
img_dir = 'linmao-test_txt/JPEGImages'
label_dir = 'linmao-test_txt/Annotations'
save_dir = 'linmao-test_txt/Masks'
save_txt_dir = 'linmao-test_txt/Masks_txt'
create_rand_mask(img_dir, label_dir, save_dir, save_txt_dir, num_per_img=4,
                     width_minmax=(0.05, 0.2),
                     hight_minmax=(0.05, 0.2),
                     img_type='jpg',
                     save_type='jpg')

In [ ]:
from object_detect.box_image_text import plot_box

In [ ]:
out_put_dir = 'linmao-test_txt/masks_on_origin'
os.makedirs(out_put_dir, exist_ok=True)
plot_box(img_dir, save_txt_dir, out_put_dir, thickness=20, filter_diff=True)

# Note
### Put mask_type as 3 and mask_file in order to use my own masks, the image name and mask name should match

In [ ]:
!python train.py --name linmao --img_file linmao-test_txt/JPEGImages --mask_file linmao-test_txt/Masks --niter 100000 --save_iters_freq 100000 --save_latest_freq 1 --batchSize 4 --mask_type 3



In [ ]:
# !python test.py -h

# batchSize must be what in order to get correct predictions
# Modify Pluralistic/options/test_options.py how_many to set up how many pictures to predict

In [ ]:
!python test.py  --name linmao --img_file linmao-test_txt/JPEGImages --mask_file linmao-test_txt/Masks --mask_type 3 --batchSize 1


In [ ]:
import torchvision.transforms as transforms
from PIL import Image, ImageFile

In [ ]:
loadSize = [522, 522]
fineSize = [512, 512]

In [ ]:
transform_list = []
osize = [loadSize[0], loadSize[1]]
fsize = [fineSize[0], fineSize[1]]

transform_list.append(transforms.Resize(fsize))

transform_list += [transforms.ToTensor()]
transform = transforms.Compose(transform_list)

In [ ]:
img_path = 'linmao-test_txt/JPEGImages/nl_0438_0420.jpg'
img_pil = Image.open(img_path).convert('RGB')
img = transform(img_pil)

In [ ]:
import numpy as np
np.array(img_pil).shape

In [ ]:
img.shape

In [ ]:
import cv2


image = img.numpy()
image = cv2.cvtColor(image)

pixels = np.array(image)

plt.imshow(pixels)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
img = img.numpy()
# img = img.reshape(img.shape[1], img.shape[2], img.shape[0])
from matplotlib import pyplot as plt
plt.imshow(img, interpolation='nearest')